In [3]:
import numpy as np
from PIL import Image
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
img_size = 64  # smaller for synthetic
categories = ["cats", "dogs"]
data = []
labels = []

def make_stripes(size=64):
    """Synthetic 'cat' image = stripes"""
    img = np.zeros((size, size))
    img[::2, :] = 255
    return img

def make_checkerboard(size=64):
    """Synthetic 'dog' image = checkerboard"""
    img = np.indices((size, size)).sum(axis=0) % 2 * 255
    return img

print("Generating synthetic dataset...")
for i in range(100):
    cat_img = make_stripes(img_size)
    dog_img = make_checkerboard(img_size)
    cat_img = cat_img + np.random.randint(0, 50, cat_img.shape)
    dog_img = dog_img + np.random.randint(0, 50, dog_img.shape)
    for img, label in [(cat_img, 0), (dog_img, 1)]:
        features = hog(img, orientations=9, pixels_per_cell=(8,8),
                       cells_per_block=(2,2), block_norm='L2-Hys',
                       visualize=False)
        data.append(features)
        labels.append(label)

X = np.array(data)
y = np.array(labels)

print(f"Synthetic dataset ready: {len(X)} samples")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print("Training SVM...")
model = SVC(kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=categories))
def predict_image(img_array):
    features = hog(img_array, orientations=9, pixels_per_cell=(8,8),
                   cells_per_block=(2,2), block_norm='L2-Hys',
                   visualize=False)
    features = scaler.transform(features.reshape(1, -1))
    prediction = model.predict(features)
    return categories[prediction[0]]
test_img = make_stripes(img_size)
print("\nPrediction for a test synthetic image:", predict_image(test_img))


Generating synthetic dataset...
Synthetic dataset ready: 200 samples
Training SVM...

Accuracy: 0.5

Classification Report:
               precision    recall  f1-score   support

        cats       0.60      0.50      0.55        24
        dogs       0.40      0.50      0.44        16

    accuracy                           0.50        40
   macro avg       0.50      0.50      0.49        40
weighted avg       0.52      0.50      0.51        40


Prediction for a test synthetic image: dogs
